In [ ]:
# Version mit vier Punkten in zwei Dimensionen und erzwungener Symmetrie

# 1. Funktionen und Bibliotheken

# Löscht alle Variablen aus der aktuellen Sitzung um sicherzustellen, 
# dass keine alten Werte oder Definitionen den Code beeinflussen.
%reset -f

# Importieren der benötigten Bibliotheken
import numpy as np  # Für numerische Berechnungen
import pandas as pd  # Für die Arbeit mit Tabellen Datensätzen
import matplotlib.pyplot as plt # Für graphische Darstellung
from sklearn.model_selection import train_test_split  # Für das Teilen der Daten in Trainings- und Testsets
from sklearn.linear_model import LinearRegression  # Für die lineare Regressionsanalyse
from sklearn.metrics import mean_squared_error  # Für die Berechnung der mittleren quadratischen Abweichung (MSE)
import cvxpy as cp # Für lineare regression mit symmetrischen Gewichten

# Definition einer Funktion für die Berechnung von Punkten auf einer kubischen Bézierkurve
def cubic_bezier(points, t):
    # Berechnet einen Punkt auf der Bézierkurve für den gegebenen Parameter t
    return ((1 - t)**3 * points[0] +  # Einfluss des ersten Kontrollpunkts
            3 * (1 - t)**2 * t * points[1] +  # Einfluss des zweiten Kontrollpunkts
            3 * (1 - t) * t**2 * points[2] +  # Einfluss des dritten Kontrollpunkts
            t**3 * points[3])  # Einfluss des vierten Kontrollpunkts

# Funktion zur Generierung von 100 Punkten auf einer Bézierkurve
def generate_bezier_points(points, num_points=100):
    # Generiert num_points Punkte auf der Bézierkurve
    return np.array([cubic_bezier(points, t) for t in np.linspace(0, 1, num_points)])

# Funktion zur Berechnung der Länge einer Kurve
def curve_length(curve):
    # Berechnet die Gesamtlänge einer Kurve basierend auf den Distanzen zwischen aufeinanderfolgenden Punkten
    distances = np.sqrt(np.sum(np.diff(curve, axis=0)**2, axis=1))
    return np.sum(distances)

# Funktion zur Berechnung der Distanz zwischen zwei Punkten
def point_distance(p1, p2):
    p1 = np.array(p1)  # Umwandeln der Punkte in numpy Arrays
    p2 = np.array(p2)
    # Berechnung der euklidischen Distanz
    return np.sqrt(np.sum((p1 - p2)**2))

In [ ]:
# 2. Bézierkurven zufallsgenerieren und vermessen
# Dies erzeugt die Daten für späteres Maschinenlernen
# n kann angepasst werden aber bei großem n dauert es lange zu rendern

# Anzahl der Bézierkurven die erzeugt werden sollen
n = 1000 

# Generierung von zufälligen Kontrollpunkten für n Bézierkurven
# Jede Bézierkurve hat 4 Kontrollpunkte im 2D-Raum
punkte_cubic = np.random.randint(low=0, high=30, size=(n, 4, 2))

# Initialisierung einer Liste zum Speichern der Ergebnisse
data = []

# Schleife über jede Gruppe von Kontrollpunkten
for i in range(n):
    control_points = punkte_cubic[i]  # Die Kontrollpunkte der aktuellen Bézierkurve
    bezier_points = generate_bezier_points(control_points)  # Generierung der Bézierkurve
    length = curve_length(bezier_points)  # Berechnung der Länge der Bézierkurve
    
    # Berechnung der Distanzen zwischen den Kontrollpunkten
    d01 = point_distance(control_points[0], control_points[1])  # Distanz zwischen P0 und P1
    d02 = point_distance(control_points[0], control_points[2])  # Distanz zwischen P0 und P2
    d03 = point_distance(control_points[0], control_points[3])  # Distanz zwischen P0 und P3
    d12 = point_distance(control_points[1], control_points[2])  # Distanz zwischen P1 und P2
    d13 = point_distance(control_points[1], control_points[3])  # Distanz zwischen P1 und P3
    d23 = point_distance(control_points[2], control_points[3])  # Distanz zwischen P2 und P3
    
    # Speichern der Ergebnisse als Eintrag in der Tabelle
    entry = {
        'P0': control_points[0].tolist(),  # Koordinaten von P0
        'P1': control_points[1].tolist(),  # Koordinaten von P1
        'P2': control_points[2].tolist(),  # Koordinaten von P2
        'P3': control_points[3].tolist(),  # Koordinaten von P3
        'Length': length,  # Länge der Bézierkurve
        'D_P0_P1': d01,  # Distanz zwischen P0 und P1
        'D_P0_P2': d02,  # Distanz zwischen P0 und P2
        'D_P0_P3': d03,  # Distanz zwischen P0 und P3
        'D_P1_P2': d12,  # Distanz zwischen P1 und P2
        'D_P1_P3': d13,  # Distanz zwischen P1 und P3
        'D_P2_P3': d23,  # Distanz zwischen P2 und P3
    }
    data.append(entry)  # Hinzufügen des Eintrags

# Erstellen eines Pandas DataFrame aus der Ergebnissammlung
df_trainingsdaten = pd.DataFrame(data)

print("Die Kontrollpunkte der Kurven, Länge der Kurven, und Abstände zwischen den Kontrollpunkten:\n")
print(df_trainingsdaten)

In [ ]:
# 3. Plotten der ersten 15 zufallsgenerierten Bézierkurven zur Ansicht

fig, axes = plt.subplots(3, 5, figsize=(20, 12))  # 3x5 Raster für 15 Kurven
axes = axes.flatten()  # Umwandeln der Achsen in Eindimensionalität für einfachere Iteration

# Für die ersten 15 Bézierkurven
for idx, control_points in enumerate(punkte_cubic[:15]):  # Nur die ersten 15 Kontrollpunkt-Sets
    bezier_curve = generate_bezier_points(control_points)  # Punkte auf der Bézierkurve generieren
    
    ax = axes[idx]
    control_points = np.array(control_points)  # Sicherstellen, dass Kontrollpunkte ein NumPy-Array sind
    
    # Plot der Bézierkurve
    ax.plot(bezier_curve[:, 0], bezier_curve[:, 1], color='blue')  # Blaue Bézierkurve
    # Kontrollpunkte plotten
    ax.plot(control_points[:, 0], control_points[:, 1], 'ro--', markersize=5)  # Rote Kontrollpunkte
    ax.set_title(f"Kurve {idx + 1}", fontsize=15)  
    ax.tick_params(axis='both', labelsize=15)  # Schriftgröße für die Achsenticks

# Entfernen überschüssiger Subplots, falls weniger als 15 Kurven vorhanden
for ax in axes[len(punkte_cubic[:15]):]:
    ax.axis('off')

# Globale Legende hinzufügen
fig.legend(
    ['Bézierkurve', 'Kontrollpunkte'], 
    loc='lower center', 
    ncol=2, 
    fontsize=20, 
    frameon=False
)

# Layout anpassen und anzeigen
plt.tight_layout(rect=[0, 0.05, 1, 1])  # Platz für die globale Legende unten schaffen
plt.show()

In [ ]:
# 4. Modell fitten und ausgeben 
# mit einigen Zusatzinformationen zur besseren Beurteilung

# Bestimmen und Ausgabe der kürzesten und längsten Länge, Durchschnitt und Median
min_length = df_trainingsdaten['Length'].round(5).min()
max_length = df_trainingsdaten['Length'].round(5).max()
mean_length = df_trainingsdaten['Length'].mean().round(5)
median_length = df_trainingsdaten['Length'].median()  # Ohne Rundung, falls nicht nötig

print(
    f"Längen der Kurven: [{min_length}, {max_length}]\n"
    f"Mittelwert: {mean_length}\n"
    f"Median: {median_length}\n"
)

# Features (X) und Zielwerte (y)
X = df_trainingsdaten[['D_P0_P1', 'D_P0_P2', 'D_P0_P3', 'D_P1_P2', 'D_P1_P3', 'D_P2_P3']].values
y = df_trainingsdaten['Length'].values

# Aufteilen der Daten in Trainings- und Testdatensätze
# 80% der Daten werden für das Training verwendet, 20% für das Testen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Anzahl der Features
n_features = X_train.shape[1]

# Definieren der Optimierungsvariablen (Gewichte)
w = cp.Variable(n_features)  # Gewichte w1, w2, ..., w6

# Symmetrieeinschränkungen: w1 = w6 und w2 = w5
constraints = [
    w[0] == w[5], 
    w[1] == w[4], 
]

# Vorhersagen für die Trainingsdaten
y_pred_train = X_train @ w

# Zielfunktion: Minimierung der mittleren quadratischen Abweichung (MSE)
objective = cp.Minimize(cp.sum_squares(y_train - y_pred_train))

# Problem definieren und lösen
problem = cp.Problem(objective, constraints)
problem.solve()

# Berechnung des MSE (Mean Squared Error) auf den Trainingsdaten
mse_train = mean_squared_error(y_train, y_pred_train.value)  # y_pred.value gibt die optimierten Vorhersagen
print(f"Mean Squared Error (Trainingsdaten): {mse_train}\n")  # Ausgabe der MSE für Trainingsdaten

# Vorhersagen auf den Testdaten
y_pred_test = X_test @ w

# Umwandlung in ein NumPy Array, falls notwendig
y_pred_test_value = y_pred_test.value  # cvxpy Variable zu NumPy Array

# Berechnung des MSE (Mean Squared Error) auf den Testdaten
mse_test = mean_squared_error(y_test, y_pred_test_value)  # Berechnung des MSE für Testdaten
print(f"Mean Squared Error (Testdaten): {mse_test}\n")  # Ausgabe der MSE für Testdaten

# Gewichte aus dem cvxpy-Problem
coefficients = w.value

# Features und Labels
features = ['D_P0_P1', 'D_P0_P2', 'D_P0_P3', 'D_P1_P2', 'D_P1_P3', 'D_P2_P3']
labels = ['a', 'b', 'c', 'd', 'e', 'f']

# Erstellen eines DataFrames für die Modellkoeffizienten
coef_df = pd.DataFrame({
    'Variable': labels,
    'Gewicht': coefficients.round(5),
    'Feature': features
})

# Ausgabe der Modellkoeffizienten
print("Gewichte der Linearen Regression:")
print(coef_df)

In [ ]:
# 5. Plotten der realen Werte vs. der Vorhersage für die Testdaten

# Anlage des Diagramms
plt.figure(figsize=(10, 6)) 
plt.scatter(y_test, y_pred_test.value, alpha=0.6, color='blue', label='Datenpunkte')  # Streudiagramm mit Testdaten

# Hinzufügen der Diagonalen für die Vorhersage
min_value = min(y_test.min(), y_pred_test.value.min(), 0)  # Kleinster Wert in den Testdaten
max_value = max(y_test.max(), y_pred_test.value.max())  # Größter Wert in den Testdaten
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='Vorhersage')

# Achsenbeschriftungen und Titel
plt.xlabel('Tatsächliche Werte (y_test)', fontsize=12)
plt.ylabel('Vorhergesagte Werte (y_pred_test)', fontsize=12)
plt.title('Berechnete vs. vorhergesagte Werte (Testdaten)', fontsize=14)

# Hinzufügen einer Legende und eines Gitters
plt.legend()
plt.grid(True)

# Plot anzeigen
plt.show()

In [ ]:
# 6. Einführen und Plotten von Testdaten aus realen Projekten
# Siehe https://github.com/benjaminwand/cookie-cutters

punkte_testdaten = [
# corona virus   
[[35, 0], [35, 4.95], [33.9, 5.5], [32.8, 5.5]], 
[[23.835, 2.75], [31.7, 2.75], [29.5, 5.5], [32.8, 5.5]],
# chaosknoten    
[[36.0, 52.0], [60.0, 51.0], [65.0, 53.0], [70.0, 47.0]],
 [[36.0, 48.9], [60.0, 49.0], [65.0, 50.0], [69.0, 47.0]],
 [[77.0, 32.4], [75.5, 28.0], [72.0, 27.0], [71.7, 21.7]],
 [[79.0, 8.1], [72.5, 15.0], [82.0, 24.0], [77.0, 32.4]],
# cactus    
[[51, 81.5], [58, 81.5], [58.5, 63], [58.5, 58]], 
[[28, 0], [20, 0], [18.5, 2.8], [18.5, 3.2]],
[[18.5, 3.2], [18.5, 4], [20.5, 10], [20.5, 24]],
[[0, 52], [0, 65], [2.5, 75.5], [9.5, 75.5]],
[[9.5, 75.5], [19.5, 75.5], [11, 46], [17.2, 46]],
[[17.2, 46], [23, 46], [14, 90], [30, 90]],
# fairydust rocket
[[20, 0], [20, 14.5], [20, 21], [7.8, 28.2]],
[[33.7, 0], [33.7, 25], [32, 43], [19.5, 51]],
[[19.5, 51], [25, 88], [15, 97], [0, 103.8]]
]

# Plotten der Bézierkurven aus den punkte_testdaten
fig, axes = plt.subplots(3, 5, figsize=(20, 12))  # 3x5 Raster für 15 Kurven
axes = axes.flatten()  # Achsen in flache Liste umwandeln

# Plot jeder Bézierkurve
for idx, control_points in enumerate(punkte_testdaten):
    bezier_curve = generate_bezier_points(np.array(control_points))  # Bézierkurve berechnen
    control_points = np.array(control_points)  # Kontrollpunkte in NumPy-Array umwandeln
    
    ax = axes[idx]
    # Bézierkurve plotten
    ax.plot(bezier_curve[:, 0], bezier_curve[:, 1], color="blue")
    # Kontrollpunkte plotten
    ax.plot(control_points[:, 0], control_points[:, 1], 'ro--', markersize=5)
    # Titel für jede Kurve
    ax.set_title(f"Bézierkurve {idx + 1}", fontsize=15)
    ax.tick_params(axis='both', labelsize=15)  # Schriftgröße für die Achsenticks

# Überschüssige Subplots deaktivieren (falls weniger als 15 Kurven)
for ax in axes[len(punkte_testdaten):]:
    ax.axis('off')

# Globale Legende hinzufügen
fig.legend(
    ['Bézierkurve', 'Kontrollpunkte'], 
    loc='lower center', 
    ncol=2, 
    fontsize=20, 
    frameon=False
)

# Layout anpassen und anzeigen
plt.tight_layout(rect=[0, 0.05, 1, 1])  # Platz für die globale Legende unten schaffen
plt.show()

In [ ]:
# 7. Berechnung und Ausgabe der Abstände zwischen den Kontrollpunkten für den Testdatensatz

test_daten = []
for control_points in punkte_testdaten:

    # Kontrollpunkte in NumPy-Array konvertieren
    control_points = np.array(control_points)

    bezier_points = generate_bezier_points(control_points)  # Generierung der Bézierkurve
    length = curve_length(bezier_points)  # Berechnung der Länge der Bézierkurve
    
    d01 = point_distance(control_points[0], control_points[1])  # Distanz zwischen P0 und P1
    d02 = point_distance(control_points[0], control_points[2])  # Distanz zwischen P0 und P2
    d03 = point_distance(control_points[0], control_points[3])  # Distanz zwischen P0 und P3
    d12 = point_distance(control_points[1], control_points[2])  # Distanz zwischen P1 und P2
    d13 = point_distance(control_points[1], control_points[3])  # Distanz zwischen P1 und P3
    d23 = point_distance(control_points[2], control_points[3])  # Distanz zwischen P2 und P3
    
    # Speichern der Ergebnisse als Eintrag
    entry = {
        'D_P0_P1': d01,   # Distanz zwischen P0 und P1
        'D_P0_P2': d02,   # Distanz zwischen P0 und P2
        'D_P0_P3': d03,   # Distanz zwischen P0 und P3
        'D_P1_P2': d12,   # Distanz zwischen P1 und P2
        'D_P1_P3': d13,   # Distanz zwischen P1 und P3
        'D_P2_P3': d23,   # Distanz zwischen P2 und P3
        'Length': length, # Länge der Bézierkurve
    }
    test_daten.append(entry)

# Umwandeln der Testdaten in ein DataFrame
df_test = pd.DataFrame(test_daten)

# Berechnung und Hinzufügen der Vorhersagen für jede Zeile im DataFrame df_test
df_test['Predicted_Length'] = (
    df_test[features].dot(coefficients) 
)

# Ausgabe des DataFrame
print("Die Kontrollpunkte der Kurven, Länge der Kurven, und Abstände zwischen den Kontrollpunkten:\n")
print(df_test)

In [ ]:
# 8. Ausgabe der mittleren quadratischen Abweichung 
# mit einigen Zusatzinformationen zur besseren Beurteilung

# Bestimmen und Ausgabe der kürzesten und längsten Länge
min_length = df_test['Length'].round(4).min()    # Kürzeste Länge
max_length = df_test['Length'].round(4).max()    # Längste Länge
mean_length = df_test['Length'].mean().round(4)  # Durchschnitt
median_length = df_test['Length'].round(4).median()  # Median

print(
    f"Längen der Kurven: [{min_length}, {max_length}]\n"
    f"Mittelwert: {mean_length}\n"
    f"Median: {median_length}\n"
)

# Vergleich der vorhergesagten und tatsächlichen Längen
mse = mean_squared_error(df_test['Length'], df_test['Predicted_Length'])
print(f"Mean Squared Error (MSE): {mse}")

In [ ]:
# 9. Erstellen des Plots, der Vorhersage und Testdaten gegenüberstellt

plt.figure(figsize=(10, 6))
plt.scatter(df_test['Length'], df_test['Predicted_Length'], color='blue', alpha=0.6, label='Berechnete Werte')

# Hinzufügen der Diagonalen für perfekte Vorhersage
max_value = max(df_test['Length'].max(), df_test['Predicted_Length'].max())
min_value = min(df_test['Length'].min(), df_test['Predicted_Length'].min(), 0)
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='Vorhersage')

# Achsenbeschriftungen und Titel
plt.xlabel('Tatsächliche Länge (Length)', fontsize=12)
plt.ylabel('Vorhergesagte Länge (Predicted Length)', fontsize=12)
plt.title('Vergleich: Tatsächliche vs. vorhergesagte Länge', fontsize=14)
plt.legend()
plt.grid(True)

# Plot anzeigen
plt.show()